import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

%matplotlib inline

import numpy as np
import math

from scipy.io.wavfile import read
from scipy.signal import spectrogram

# 1. Serviços de Sinal Horário

Na Europa, existem dois transmissores de sinal horario que operam com frequâncias suficientemente baixas para conseguirmos registar o seu sinal. São eles o MSF, localizado no Reino Unido e que opera a $60kHz$, e o DCF77, localizado na Alemanha e a operar a $77.5kHz$.

In [ ]:
plt.figure(figsize=(8,8))

lat2 = 58
lat1 = 32.
lon2 = 26.
lon1 = -15.

m = Basemap(resolution='l',projection='merc', llcrnrlat=lat1,urcrnrlat=lat2,llcrnrlon=lon1,urcrnrlon=lon2)

m.drawcountries(linewidth=0.6,color=(0.098039, 0.098039, 0.439216))
m.drawcoastlines(linewidth=0.6,color=(0.098039, 0.098039, 0.439216))
m.drawparallels(np.arange(lat1,lat2,10.),labels=[1,0,0,0],color='black',dashes=[1,2],labelstyle='+/-',linewidth=0.2)  
m.drawmeridians(np.arange(lon1,lon2,10.),labels=[0,0,0,1],color='black',dashes=[1,2],labelstyle='+/-',linewidth=0.2)

lons = [-8.112222,-3.279302 ,9.008307 ]
lats = [40.883333,54.911195 ,50.015411]
nomes=['Arada','MSF','DCF77']

m.drawgreatcircle(lons[0],lats[0],lons[1], lats[1], del_s=100.0, linewidth=1.5,color=(0.6 ,0,0))
m.drawgreatcircle(lons[0],lats[0],lons[2], lats[2], del_s=100.0, linewidth=1.5,color=(0.6 ,0,0))

x,y = m(lons,lats)

m.plot(x,y,'o',markersize=5, color=(0.6,0,0),markeredgewidth=0.01)

plt.text(x[0]+10000,y[0]-100000,nomes[0])
plt.text(x[1]+10000,y[1]-100000,nomes[1])
plt.text(x[2]+10000,y[2]-100000,nomes[2])

plt.draw()
#plt.savefig("RadioTime.png", bbox_inches='tight', pad_inches=0.55, dpi=300)
#plt.show()

# 2. Distâncias e Azimute

Assumindo um modelo esférico para a Terra, a distância entre quaisquer dois pontos:$P_1=(lat_1,log_1)$ e $P_2=(lat_2,log_2)$ , é dada por:
\begin{equation}
D= R \arccos(\sin(lat_1)\sin(lat_2)+\cos(lat_1)\cos(lat_2)\cos(long_2-long_1))
\end{equation}
sendo $R=6371km$, o raio médio da Terra.

Ver também o [link](http://www.movable-type.co.uk/scripts/latlong.html).

O Azimute, em radianos, pode ser calculado através de:

$$\theta=\arctan 2(Y,X),
$$

onde 
$$
X=\cos(lat_1)\sin(lat_2)-\sin(lat_1)\cos(lat_2) \cos(lon_2-lon_1)
$$
e
$$
Y= \sin(lon_2-lon_1) \cos(lat_2),
$$

onde consideramos o $P_1$ o ponto onde se localiza o nosso observatório.

In [ ]:
def distancia(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    rlat1 = math.radians(lat1)
    rlon1 = math.radians(lon1)
    rlat2 = math.radians(lat2)
    rlon2 = math.radians(lon2)
    
    a = math.sin(rlat1) * math.sin(rlat2) + math.cos(rlat1)*math.cos(rlat2)*math.cos(rlon2-rlon1)
    d = radius * math.acos(a)
    print'A  distância entre os dois pontos é de %fkm' %d
    return d

In [ ]:
def Azimute(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination

    rlat1 = math.radians(lat1)
    rlon1 = math.radians(lon1)
    rlat2 = math.radians(lat2)
    rlon2 = math.radians(lon2)
    
    X = math.cos(rlat1) * math.sin(rlat2) - math.sin(rlat1)*math.cos(rlat2)*math.cos(rlon2-rlon1)
    Y = math.sin(rlon2-rlon1)*math.cos(rlat2)
    a = math.degrees(math.atan2(Y,X))
    print'O Azimute entre os dois pontos é de %f°' %a

    return a

In [ ]:
Arada = (40.883333,-8.112222)
MSF = (54.911195,-3.279302)
DCF77 = (50.015411,9.008307)

## 2.1 MSF

In [ ]:
d1 = distancia(Arada, MSF)

In [ ]:
a1 = Azimute(Arada, MSF)

## 2.2 DCF77

In [ ]:
d2 = distancia(Arada, DCF77)

In [ ]:
a2 = Azimute(Arada, DCF77)

# 3. Codificação do sinal horário

<img src='logfile1_02_02_2011OVERLAP.png'>

In [ ]:
(sample_rate, input_signal) = read("data/Avelinha-5-5-2017-I.wav")
f, t, Sxx = spectrogram(np.sum(input_signal, axis=1),
                        sample_rate, nperseg=512, nfft=4096)


## 3.1 MSF

Para mais informação ver os segintes links: [links1](http://www.npl.co.uk/science-technology/time-frequency/)

## 3.2 DSF77

Para mais informação ver os segintes links: [links](https://en.wikipedia.org/wiki/DCF77),   [links](http://www.ptb.de/cms/en/ptb/fachabteilungen/abt4/fb-44/ag-442/dissemination-of-legal-time/dcf77.html)

Links:
1. [ Sudden Ionospheric Disturbances Monitoring Station A118](https://sidstation.loudet.org/)

2. [vlf.it](http://www.vlf.it/)

3. [Construção de recetores rádio como introdução à Física das Telecomunicações - parte I ](https://www.spf.pt/magazines/GFIS/113/913)

4. [Construção de recetores rádio como introdução à Física das Telecomunicações - parte II ](https://www.spf.pt/magazines/GFIS/114/931)

5. [Ouvindo o emissor SAQ de Grimeton ](https://www.spf.pt/magazines/GFIS/118/969)